In [46]:
from datasets import load_dataset

dataset = load_dataset("Atomi/semeval_2013_task_7_beetle_5way")

In [83]:
import dspy
import openai

gpt3_turbo_instruct = dspy.OpenAI(model="gpt-3.5-turbo-instruct", api_key="sk-proj-5V1twRoUPNj66jRsIu57T3BlbkFJOG9TKDA7jQHMNQj9MVZM", max_tokens=1000)
llama3_ollama = dspy.OllamaLocal(model="llama3", max_tokens=1000)
# lm_llama3_HF = dspy.HFModel(model="meta-llama/Meta-Llama-3-8B-Instruct")
# https://dspy-docs.vercel.app/api/local_language_model_clients/HFModel

dspy.settings.configure(lm=llama3_ollama)

In [69]:
dataset["train"][0]

{'question_id': 'HYBRID_BURNED_OUT_EXPLAIN_Q1',
 'question': 'Explain your reasoning.',
 'question_qtype': 'Q_EXPLAIN_SPECIFIC',
 'question_module': 'SwitchesBulbsParallel',
 'question_stype': 'PREDICT',
 'reference_answer': 'If bulb A burns out, B and C are no longer in a closed path with the battery',
 'reference_answer_quality': 'BEST',
 'student_answer': 'bulb b and c will not be in a closed path with the battery',
 'label_5way': 'correct',
 'test_set': 'training'}

In [70]:
class AssessmentSignature(dspy.Signature):
    """
    Role: Academic Tutor
    Task: Make an assessment of the student's answer by comparing it with the reference answer.
    You are essentially assessing whether the student's answer was correct, or needs additional feedback.
    
    The value for assessment must be EXACTLY one of the following classes:
    "correct", "partially_correct_incomplete", "contradictory", "non_domain", "irrelevant"
    Include the underscores if the class contains it.

    Definitions of the classes:
    - correct: if the student answer is a complete and correct paraphrase of the reference answer
    - partially_correct_incomplete: if it is a partially correct answer containing some but not
    all information from the reference answer
    - contradictory: if the student answer explicitly contradicts the reference answer
    - irrelevant:  if the student answer is talking about domain content but not providing the necessary information
    - non_domain: if the student utterance does not include domain content, e.g., “I don’t know”, “what the book says”, “you are stupid”

    The "question" field doesn't provide too much context apart from the final few words of the problem.
    So don't focus on the question too much.
    """

    question: str = dspy.InputField()
    student_answer: str = dspy.InputField()
    reference_answer: str = dspy.InputField()

    assessment: str = dspy.OutputField(desc="the final assessment")

class ASAGCoT(dspy.Module):
    """Assess the student's answer to the question by comparing with the reference answer"""

    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(AssessmentSignature)

    def forward(self, question: str, student_answer: str, reference_answer: str):
        output = self.prog(question=question, student_answer=student_answer, reference_answer=reference_answer)
        
        output.assessment = str(output.assessment).lower()
        dspy.Suggest(
            output.assessment in ["correct", "partially_correct_incomplete", "contradictory", "non_domain", "irrelevant"],
            f"'{output.assessment}' must be exactly one of 'correct', 'partially_correct_incomplete', 'contradictory', 'non_domain', 'irrelevant'"
        )
        
        return output

## Trying Zero shot

In [71]:
with dspy.context(lm=gpt3_turbo_instruct):
    asag_cot_pipeline = ASAGCoT()

question, s_answer, r_answer = dataset["train"][0]["question"], dataset["train"][0]["student_answer"], dataset["train"][0]["reference_answer"]
assessment = asag_cot_pipeline.forward(question, s_answer, r_answer)

In [72]:
question

'Explain your reasoning.'

In [73]:
s_answer

'bulb b and c will not be in a closed path with the battery'

In [74]:
r_answer

'If bulb A burns out, B and C are no longer in a closed path with the battery'

In [75]:
assessment

Prediction(
    rationale="Here is the assessment:\n\n---\nQuestion: Explain your reasoning.\nStudent Answer: bulb b and c will not be in a closed path with the battery\nReference Answer: If bulb A burns out, B and C are no longer in a closed path with the battery\nReasoning: Let's think step by step in order to assess whether the student's answer is correct or needs additional feedback. The student's answer states that bulbs B and C will not be in a closed path with the battery when bulb A burns out, which is partially correct but incomplete. The reference answer explicitly states the condition under which bulbs B and C are no longer in a closed path with the battery, which is if bulb A burns out. Therefore, the student's answer is partially correct but incomplete.",
    assessment='partially_correct_incomplete'
)

In [78]:
dataset["train"][0]["label_5way"]

'correct'

In [79]:
gpt3_turbo_instruct.inspect_history(n=1)

''

## Creating a Dataset

In [90]:
test_set = dataset["test"]

dspy_test_dataset = [
    dspy.Example(
        question=data["question"],
        student_answer=data["student_answer"],
        reference_answer=data["reference_answer"],
        assessment=data["label_5way"]
    ).with_inputs("question","student_answer","reference_answer")
    for data in test_set
]

In [91]:
print(f"we have {len(dspy_test_dataset)} test data points in total")

we have 1489 test data points in total


In [80]:
train_set = dataset["train"]

best_quality_dataset = [example for example in train_set if example["reference_answer_quality"] == "BEST"]

dspy_dataset_best_quality_all = [
    dspy.Example(
        question=data["question"],
        student_answer=data["student_answer"],
        reference_answer=data["reference_answer"],
        assessment=data["label_5way"]
    ).with_inputs("question","student_answer","reference_answer")
    for data in best_quality_dataset
]

In [81]:
dspy_dataset_best_quality_all

[Example({'question': 'Explain your reasoning.', 'student_answer': 'bulb b and c will not be in a closed path with the battery', 'reference_answer': 'If bulb A burns out, B and C are no longer in a closed path with the battery', 'assessment': 'correct'}) (input_keys={'student_answer', 'reference_answer', 'question'}),
 Example({'question': 'Explain your reasoning.', 'student_answer': 'there is no closed path that contains the battery, bulb b, and bulb c', 'reference_answer': 'If bulb A burns out, B and C are no longer in a closed path with the battery', 'assessment': 'correct'}) (input_keys={'student_answer', 'reference_answer', 'question'}),
 Example({'question': 'Explain your reasoning.', 'student_answer': 'creates a gap', 'reference_answer': 'If bulb A burns out, B and C are no longer in a closed path with the battery', 'assessment': 'partially_correct_incomplete'}) (input_keys={'student_answer', 'reference_answer', 'question'}),
 Example({'question': 'Explain your reasoning.', 'stu

In [59]:
print(f"we have {len(dspy_dataset_best_quality_all)} data points in total")

we have 5129 data points in total


In [60]:
# creating a smaller dataset, with 60 examples for each label
import random

num_examples_per_label = 60

# Initialize lists to store examples for each label
correct_examples = []
partially_correct_examples = []
contradictory_examples = []
non_domain_examples = []
irrelevant_examples = []

# Iterate through the original dataset and filter examples by label
for example in best_quality_dataset:
    label = example["label_5way"]
    if label == "correct":
        correct_examples.append(example)
    elif label == "partially_correct_incomplete":
        partially_correct_examples.append(example)
    elif label == "contradictory":
        contradictory_examples.append(example)
    elif label == "non_domain":
        non_domain_examples.append(example)
    elif label == "irrelevant":
        irrelevant_examples.append(example)

# Randomly sample 60 examples for each label
sampled_examples = (
    random.sample(correct_examples, num_examples_per_label) +
    random.sample(partially_correct_examples, num_examples_per_label) +
    random.sample(contradictory_examples, num_examples_per_label) +
    random.sample(non_domain_examples, num_examples_per_label) +
    random.sample(irrelevant_examples, num_examples_per_label)
)

# Shuffle the sampled examples
random.shuffle(sampled_examples)

# Create dspy.Example objects for the sampled examples
dspy_dataset_equal_distribution = [
    dspy.Example(
        question=example["question"],
        student_answer=example["student_answer"],
        reference_answer=example["reference_answer"],
        assessment=example["label_5way"]
    ).with_inputs("question", "student_answer", "reference_answer")
    for example in sampled_examples
]

In [61]:
print(f"we have {len(dspy_dataset_equal_distribution)} data points in dspy_dataset_equal_distribution")

we have 300 data points in dspy_dataset_equal_distribution


In [62]:
from sklearn.model_selection import train_test_split

train_data, dev_data = train_test_split(dspy_dataset_equal_distribution, test_size=0.3, random_state=42)

In [65]:
train_data

[Example({'question': 'Explain why you got a voltage reading of 1.5 for terminal 1 and the positive terminal.', 'student_answer': 'the positive terminal is connected to the bulb', 'reference_answer': 'Terminal 1 and the positive terminal are not connected', 'assessment': 'irrelevant'}) (input_keys={'student_answer', 'reference_answer', 'question'}),
 Example({'question': 'Explain your reasoning.', 'student_answer': 'they have the same path', 'reference_answer': 'A and C are in the same closed path', 'assessment': 'correct'}) (input_keys={'student_answer', 'reference_answer', 'question'}),
 Example({'question': 'What role does the path play in determining whether or not a switch affects a bulb?', 'student_answer': 'if the bulb is contained within the path', 'reference_answer': 'If a bulb and a switch are in the same path the switch affects the bulb', 'assessment': 'partially_correct_incomplete'}) (input_keys={'student_answer', 'reference_answer', 'question'}),
 Example({'question': 'Whe

## Building a metric

In [66]:
import dspy

def grading_accuracy_metric(example, predicted, trace=None):
    
    score = 1 if example["assessment"] == predicted["assessment"] else 0
    
    return score

from dspy.evaluate import Evaluate

EM_metric = dspy.evaluate.answer_exact_match

## Optimising Few Shot Examples

In [84]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFewShot

# with dspy.context(lm=gpt3_turbo_instruct):
dspy.settings.configure(lm=gpt3_turbo_instruct)

config = dict(max_bootstrapped_demos=10, max_labeled_demos=30, num_candidate_programs=4, num_threads=2)
optimiser = BootstrapFewShotWithRandomSearch(metric=grading_accuracy_metric, **config)
# optimiser = BootstrapFewShot(metric=grading_accuracy, **config)
optimised_program = optimiser.compile(asag_cot_pipeline, trainset=train_data)

Average Metric: 48 / 150  (32.0):  71%|▋| 150/210 [01:58<01:01,  1.02s/itERROR:dspy.primitives.assertions:2024-05-17T15:10:04.854661Z [error    ] SuggestionFailed: 'partially_correct_incomplete_contradictory' must be exactly one of 'correct', 'partially_correct_incomplete', 'contradictory', 'non_domain', 'irrelevant' [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR:dspy.evaluate.evaluate:2024-05-17T15:10:04.857115Z [error    ] Error for example in dev set: 		 'partially_correct_incomplete_contradictory' must be exactly one of 'correct', 'partially_correct_incomplete', 'contradictory', 'non_domain', 'irrelevant' [dspy.evaluate.evaluate] filename=evaluate.py lineno=147
Average Metric: 36 / 88  (40.9):  42%|█████████████████████████████████████████████████████████████████████████████▌                                                                                                           | 88/210 [01:05<01:50,  1.10it/s]INFO:backoff:Backing off request(...) for 0.2s (

Backing off 0.2 seconds after 1 tries calling function <function GPT3.request at 0x12e9b4860> with kwargs {}


Average Metric: 37 / 90  (41.1):  43%|███████████████████████████████████████████████████████████████████████████████▎                                                                                                         | 90/210 [01:08<02:18,  1.15s/it]INFO:backoff:Backing off request(...) for 1.3s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct in organization org-DNIQnZOwEXH3HneN5ZlhWRmr on tokens per min (TPM): Limit 90000, Used 88349, Requested 3364. Please try again in 1.142s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})


Backing off 1.3 seconds after 2 tries calling function <function GPT3.request at 0x12e9b4860> with kwargs {}


Average Metric: 37 / 96  (38.5):  46%|████████████████████████████████████████████████████████████████████████████████████▌                                                                                                    | 96/210 [01:16<02:19,  1.23s/it]INFO:backoff:Backing off request(...) for 0.3s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct in organization org-DNIQnZOwEXH3HneN5ZlhWRmr on tokens per min (TPM): Limit 90000, Used 88348, Requested 3370. Please try again in 1.145s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})


...

Average Metric: 78 / 187  (41.7):  89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                    | 187/210 [04:48<00:34,  1.48s/it]INFO:backoff:Backing off request(...) for 21.9s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct in organization org-DNIQnZOwEXH3HneN5ZlhWRmr on tokens per min (TPM): Limit 90000, Used 88253, Requested 3679. Please try again in 1.288s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})


Backing off 21.9 seconds after 6 tries calling function <function GPT3.request at 0x12e9b4860> with kwargs {}


Average Metric: 86 / 204  (42.2):  97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊     | 204/210 [05:14<00:08,  1.40s/it]

Backing off 57.4 seconds after 7 tries calling function <function GPT3.request at 0x12e9b4860> with kwargs {}


Average Metric: 90 / 210  (42.9): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 210/210 [06:12<00:00,  1.78s/it]


In [85]:
optimised_program

prog = ChainOfThought(AssessmentSignature(question, student_answer, reference_answer -> assessment
    instructions='\n    Role: Academic Tutor\n    Task: Make an assessment of the student\'s answer by comparing it with the reference answer.\n    You are essentially assessing whether the student\'s answer was correct, or needs additional feedback.\n    \n    The value for assessment must be EXACTLY one of the following classes:\n    "correct", "partially_correct_incomplete", "contradictory", "non_domain", "irrelevant"\n    Include the underscores if the class contains it.\n\n    Definitions of the classes:\n    - correct: if the student answer is a complete and correct paraphrase of the reference answer\n    - partially_correct_incomplete: if it is a partially correct answer containing some but not\n    all information from the reference answer\n    - contradictory: if the student answer explicitly contradicts the reference answer\n    - irrelevant:  if the student answer is talking ab

In [86]:
optimised_program.save("asag_cot_better_instruction_prompt.json")

In [87]:
gpt3_turbo_instruct.inspect_history(n=1)




Role: Academic Tutor
    Task: Make an assessment of the student's answer by comparing it with the reference answer.
    You are essentially assessing whether the student's answer was correct, or needs additional feedback.
    
    The value for assessment must be EXACTLY one of the following classes:
    "correct", "partially_correct_incomplete", "contradictory", "non_domain", "irrelevant"
    Include the underscores if the class contains it.

    Definitions of the classes:
    - correct: if the student answer is a complete and correct paraphrase of the reference answer
    - partially_correct_incomplete: if it is a partially correct answer containing some but not
    all information from the reference answer
    - contradictory: if the student answer explicitly contradicts the reference answer
    - irrelevant:  if the student answer is talking about domain content but not providing the necessary information
    - non_domain: if the student utterance does not include domain conte

'\n\n\nRole: Academic Tutor\n    Task: Make an assessment of the student\'s answer by comparing it with the reference answer.\n    You are essentially assessing whether the student\'s answer was correct, or needs additional feedback.\n    \n    The value for assessment must be EXACTLY one of the following classes:\n    "correct", "partially_correct_incomplete", "contradictory", "non_domain", "irrelevant"\n    Include the underscores if the class contains it.\n\n    Definitions of the classes:\n    - correct: if the student answer is a complete and correct paraphrase of the reference answer\n    - partially_correct_incomplete: if it is a partially correct answer containing some but not\n    all information from the reference answer\n    - contradictory: if the student answer explicitly contradicts the reference answer\n    - irrelevant:  if the student answer is talking about domain content but not providing the necessary information\n    - non_domain: if the student utterance does not 

## Evaluation on Dev Set

In [89]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=dev_data, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluator(optimised_program, metric=grading_accuracy_metric)

Average Metric: 33 / 90  (36.7): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [02:07<00:00,  1.42s/it]


,question,student_answer,reference_answer,example_assessment,rationale,pred_assessment,grading_accuracy_metric
0,Why does a damaged bulb impact a circuit?,a damaged bulb does not make a connecton so the circut is not complete,a damaged bulb creates a gap,correct,"produce the assessment. We can see that the student answer mentions a damaged bulb not making a connection, which is partially correct. However, it does...",partially_correct_incomplete,0
1,Explain your reasoning.,bulb B is in a separate path than bulb A and bulb C.,Bulbs A and C are still in closed paths with the battery,correct,"produce the assessment. We can see that the student answer mentions bulb B being in a separate path, which is not the same as saying...",partially_correct_incomplete,0
2,Explain why you got a voltage reading of 1.5 for terminal 1 and the positive terminal.,Terminal 1 was connected to the positive terminal.,Terminal 1 and the positive terminal are separated by the gap,contradictory,"produce the assessment. We can see that the student answer correctly identifies that terminal 1 was connected to the positive terminal, but it does not...",partially_correct_incomplete,0
3,Explain why you got a voltage reading of 0 for terminal 5 and the positive terminal.,Because. It was on the wrong side.,Terminal 5 and the positive terminal are connected,contradictory,produce the assessment. We can see that the student answer is not relevant to the question and does not provide any information about the connection...,non_domain,0
4,Under what circumstances will a switch affect a bulb?,If it is on the path with it and the battery,When the switch and the bulb are contained in the same path,partially_correct_incomplete,"produce the assessment. We can see that the student answer is partially correct, as it mentions the role of the path and the battery in...",partially_correct_incomplete,✔️ [1]


36.67

## Evaluation on Test set

In [92]:
evaluator = Evaluate(devset=dspy_test_dataset, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluator(optimised_program, metric=grading_accuracy_metric)

Average Metric: 97 / 296  (32.8):  20%|████████████████████████████████████████▌                                                                                                                                                                   | 296/1489 [08:10<33:18,  1.68s/it]

KeyboardInterrupt: 